In [3]:
from facenet.model import create_model
nn4_small2 = create_model()

In [4]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Layer

# Input for anchor, positive and negative images
in_a = Input(shape=(96, 96, 3))
in_p = Input(shape=(96, 96, 3))
in_n = Input(shape=(96, 96, 3))

# Output for anchor, positive and negative embedding vectors
# The nn4_small model instance is shared (Siamese network)
emb_a = nn4_small2(in_a)
emb_p = nn4_small2(in_p)
emb_n = nn4_small2(in_n)

class TripletLossLayer(Layer):
    def __init__(self, alpha, **kwargs):
        self.alpha = alpha
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def triplet_loss(self, inputs):
        a, p, n = inputs
        p_dist = K.sum(K.square(a-p), axis=-1)
        n_dist = K.sum(K.square(a-n), axis=-1)
        return K.sum(K.maximum(p_dist - n_dist + self.alpha, 0), axis=0)
    
    def call(self, inputs):
        loss = self.triplet_loss(inputs)
        self.add_loss(loss)
        return loss

# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=0.2, name='triplet_loss_layer')([emb_a, emb_p, emb_n])

# Model that can be trained with anchor, positive negative images
nn4_small2_train = Model([in_a, in_p, in_n], triplet_loss_layer)

In [9]:
import numpy as np

def triplet_generator():
    ''' Dummy triplet generator for API usage demo only.
    Will be replaced by a version that uses real image data later.
    :return: a batch of (anchor, positive, negative) triplets
    '''
    while True:
        a_batch = np.random.rand(4, 96, 96, 3)
        p_batch = np.random.rand(4, 96, 96, 3)
        n_batch = np.random.rand(4, 96, 96, 3)
        yield [a_batch , p_batch, n_batch], None

In [10]:
# triplet_generator() creates a generator that continuously returns 
# ([a_batch, p_batch, n_batch], None) tuples where a_batch, p_batch 
# and n_batch are batches of anchor, positive and negative RGB images 
# each having a shape of (batch_size, 96, 96, 3).
generator = triplet_generator() 

nn4_small2_train.compile(loss=None, optimizer='adam')
nn4_small2_train.fit_generator(generator, epochs=10, steps_per_epoch=100)

# Please note that the current implementation of the generator only generates 
# random image data. The main goal of this code snippet is to demonstrate 
# the general setup for model training. In the following, we will anyway 
# use a pre-trained model so we don't need a generator here that operates 
# on real training data. I'll maybe provide a fully functional generator
# later.

/home/chanranhari/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Output "triplet_loss_layer" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "triplet_loss_layer" during training.
  import sys


Epoch 1/10
100/100 [==============================] - 36s 360ms/step - loss: 0.8014
Epoch 2/10
100/100 [==============================] - 12s 120ms/step - loss: 0.8000
Epoch 3/10
100/100 [==============================] - 12s 118ms/step - loss: 0.7994
Epoch 4/10
100/100 [==============================] - 12s 118ms/step - loss: 0.7985
Epoch 5/10
100/100 [==============================] - 12s 115ms/step - loss: 0.8025
Epoch 6/10
100/100 [==============================] - 11s 113ms/step - loss: 0.8002
Epoch 7/10
100/100 [==============================] - 12s 117ms/step - loss: 0.8005
Epoch 8/10
100/100 [==============================] - 12s 116ms/step - loss: 0.8001
Epoch 9/10
100/100 [==============================] - 11s 112ms/step - loss: 0.8002
Epoch 10/10
100/100 [==============================] - 11s 114ms/step - loss: 0.8000


In [11]:
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('facenet/nn4.small2.v1.h5')